# Single-output Use Case Demonstration
##### Author: Partha Seetala

In [1]:
import numpy as np
import pandas as pd
import random
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from keras.initializers import he_normal
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [2]:
# This function generates some synthetic data {sqft, bedrooms, lotsize and price}
def generate_data(count, noutputs=1):
    sqft_ranges = [0, 1500, 2500, 3500, 10000]
    price_ranges = [800000, 1000000, 2500000, 3000000]

    x = []
    y = []

    for i in range(count):
        sqft = random.randint(1800,4500)
        nbedrooms = 1+ int(sqft/1000)
        lotsize = int(random.uniform(1.2, 2.0) * sqft)

        price = 0
        for r in range(len(sqft_ranges)-1):
            if sqft >= sqft_ranges[r] and sqft < sqft_ranges[r+1]:
                price = price_ranges[r]
                break
        assert price != 0

        lotsize_value = int(random.randint(50000, 300000) * (lotsize/sqft))
        price = price + lotsize_value
        price = round(price, -3)

        if noutputs == 1:
            x.append([sqft, nbedrooms, lotsize])
            y.append(price)
        else:
            x.append([sqft, lotsize])
            y.append([price, nbedrooms])

    return x, y

# **Single-output Regression Use Case**


---

**Given the `sqft`, `bedrooms` and `lotsize` predict the `price` of a house**

**Let's initialize a Standardization (Normalization) module that we'll use in our model**

In [3]:
norm_x = StandardScaler() # We'll use this to normalize our input
norm_y = StandardScaler() # We'll use this to normalize our output

**Let's `build` a model to predict 1 output "`price`", given a house's "`sqft`", "`lotsize`" and "`bedrooms`"**

In [4]:
def build_model_for_one_output():
    model = Sequential()

    # Hidden Layer(s)
    model.add(Dense(10, input_dim=3, kernel_initializer=he_normal(), activation="relu"))

    # Output Layer
    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

model1 = build_model_for_one_output()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Let's generate some data**

In [5]:
# Generate some house price data
x, y = generate_data(10000, noutputs=1)

# Let's display the data we generated
data = pd.DataFrame(
    {'sqft': [rec[0] for rec in x],
    'nbedrooms': [rec[1] for rec in x],
    'lotsize': [rec[2] for rec in x],
    'price': y})
data

,sqft,nbedrooms,lotsize,price
0,2076,3,2654,1283000
1,2414,3,3847,1138000
2,3924,4,5262,3161000
3,3414,4,6341,2743000
4,2709,3,4988,2728000
...,...,...,...,...
9995,2839,3,4394,2803000
9996,4073,5,5097,3078000
9997,3870,4,4726,3068000
9998,2437,3,3866,1409000


**Let's `train` our model**

In [6]:
def train_model_for_one_output(model, x, y):
    # Normalize input data
    xnorm = norm_x.fit_transform(np.array(x))
    ynorm = norm_y.fit_transform(np.array(y).reshape(-1,1))

    # Train the model
    model.fit(xnorm, ynorm, epochs=100, batch_size=10, verbose=1)

train_model_for_one_output(model1, x, y)

Epoch 1/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.2082
Epoch 2/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.1051
Epoch 3/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0988
Epoch 4/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0842
Epoch 5/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0763
Epoch 6/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0643
Epoch 7/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0587
Epoch 8/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0524
Epoch 9/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0543
Epoch 10/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0536
Epoch 11/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0540
Epoch 12/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0495
Epoch 13/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0491
Epoch 14/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0505
Epoch 15/100
10

**Let's use our model to make predictions**

In [7]:
def predict_model_for_one_output(model):
    x, yt = generate_data(20, noutputs=1)

    # We have to normalize the input again even during prediction because we normalized during training
    xnorm = norm_x.transform(np.array(x))

    predictions = model.predict(xnorm)

    # De-Normalize predictions
    yp = norm_y.inverse_transform(predictions).flatten()

    return x, yt, yp

x, yt, yp = predict_model_for_one_output(model1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


**Let's see how well our model did with the predictions**

In [8]:
def show_predictions_for_one_output(x, yt, yp):
    # Pretty Print the Predictions comparing them what they should be
    offby = []
    percent = []
    for i in range(len(yp)):
        diff = int(yt[i] - yp[i])
        offby.append(diff)
        percent.append(int(100*(diff/yt[i])))

    data = pd.DataFrame(
        {'sqft': [rec[0] for rec in x],
            'nbedrooms': [rec[1] for rec in x],
            'lotsize': [rec[2] for rec in x],
            'price': yt,
            'price_pred': [int(p) for p in yp],
            'price_offby': offby,
            'price_percent_off': percent})
    return data

show_predictions_for_one_output(x, yt, yp)

,sqft,nbedrooms,lotsize,price,price_pred,price_offby,price_percent_off
0,4043,5,6611,3360000,3291681,68318,2
1,2642,3,3380,2717000,2729364,-12364,0
2,3985,4,7181,3158000,3479672,-321672,-10
3,2493,3,4556,1358000,2058048,-700048,-51
4,2184,3,4262,1215000,1317972,-102972,-8
5,4066,5,5922,3340000,3256749,83250,2
6,1991,2,3279,1244000,1318000,-74000,-5
7,2199,3,3110,1306000,1247294,58705,4
8,4274,5,8347,3273000,3360034,-87034,-2
9,2588,3,4552,2619000,2814120,-195120,-7


**Let's initialize a Standardization module we'll use in our model**

In [9]:
norm_x = StandardScaler() # We'll use this to standardize our input
norm_y = StandardScaler() # We'll use this to standardize our output

**Let's `build` a model to predict 2 outputs**

In [10]:
def build_model_for_two_outputs():
    model = Sequential()

    # Hidden Layer(s)
    model.add(Dense(10, input_dim=2, kernel_initializer=he_normal(), activation="relu"))

    # Output Layer
    model.add(Dense(2))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

model2 = build_model_for_two_outputs()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Let's generate some data**

In [11]:

# Generate some house price data
x, y = generate_data(10000, noutputs=2)

# Let's display the data we generated
data = pd.DataFrame(
    {'X_sqft': [rec[0] for rec in x],
    'X_lotsize': [rec[1] for rec in x],
    'Y_price': [rec[0] for rec in y],
    'Y_nbedrooms': [rec[1] for rec in y]})
data

,X_sqft,X_lotsize,Y_price,Y_nbedrooms
0,3344,4481,2881000,4
1,3609,6878,3112000,4
2,3034,5810,2965000,4
3,2986,5638,2946000,3
4,4112,5564,3089000,5
...,...,...,...,...
9995,3838,4738,3303000,4
9996,4350,8699,3151000,5
9997,3279,4960,2928000,4
9998,4404,8534,3102000,5


**Let's `train` our model**

In [ ]:
def train_model_for_two_outputs(model, x, y):
    # Normalize data
    xnorm = norm_x.fit_transform(np.array(x))
    ynorm = norm_y.fit_transform(np.array(y))

    # Train the model
    model.fit(xnorm, ynorm, epochs=100, batch_size=10, verbose=1)

train_model_for_two_outputs(model2, x, y)

**Let's use our model to make predictions**

In [ ]:
def predict_model_for_two_outputs(model):
    x, yt = generate_data(20, noutputs=2)

    xnorm = norm_x.transform(np.array(x))

    predictions = model.predict(xnorm)

    # De-Normalize predictions
    yp = norm_y.inverse_transform(predictions)

    return x, yt, yp

x, yt, yp = predict_model_for_two_outputs(model2)

**Let's see what our predictions look like**

In [ ]:
def show_predictions_for_two_outputs(x, yt, yp):
    price_offby = []
    price_percent = []

    nbedrooms_offby = []
    nbedrooms_percent = []

    for i in range(len(yp)):
        price_diff = int(yt[i][0] - yp[i][0])
        price_offby.append(price_diff)
        price_percent.append(int(100*(price_diff/yt[i][0])))

        nbedrooms_diff = yt[i][1] - yp[i][1]
        nbedrooms_offby.append(nbedrooms_diff)
        nbedrooms_percent.append(int(100*(nbedrooms_diff/yt[i][1])))

    data = pd.DataFrame(
            {'sqft': [rec[0] for rec in x],
            'lotsize': [rec[1] for rec in x],
            'price': [int(p[0]) for p in yt],
            'price_pred': [int(p[0]) for p in yp],
            'price_offby': price_offby,
            'price_percent_off': price_percent,
            'nbedrooms': [p[1] for p in yt],
            'nbedrooms_pred': [int(p[1]) for p in yp],
            'nbedrooms_offby': nbedrooms_offby,
            'nbedrooms_percent_of': nbedrooms_percent})
    return data

show_predictions_for_two_outputs(x, yt, yp)